In [1]:
import pandas as pd
import glob

### 1. We need to identify a way to consistently extract features for different cities stored in multiple files.
Here, I create a list of files with fog data and extract city file names because fog data files are stored as "CityName19xx-xxxxFogData.txt". Inorder to get the city name I extract city name by considering all the string till it encounters first 1

In [10]:
fogDataFiles = glob.glob("*2016_FogData.txt")
cityNames = []
for fileName in fogDataFiles:
    cityNames.append(fileName[0:fileName.find('1')])
del fogDataFiles

### 2. I can create a fileList for different cities <font color = 'red'> and naively removing files with "xxPercentagexx" in the  list </font>

In [25]:
for city in cityNames :
    fileList = glob.glob(str('*'+ city + '*'))
    fileList = [x for x in fileList if ("Percent" not in x)]
    print([city,len(fileList)])
    dataL = pd.DataFrame()
    for fileName in fileList : 

        indexNum = 6
        if(not fileName.startswith(city)):
            varName = fileName[0:fileName.find('_')]
        else :
            varName = fileName[::-1][0:fileName[::-1].find('_')][::-1][:-4]
            if ('Fog' in varName) :
                indexNum = 0
                if('Mean' in varName):
                    varName = fileName[::-1][0:fileName[::-1].find('6')][::-1][:-4]

        dataL = pd.concat([dataL, pd.read_csv(fileName, index_col=indexNum)], axis = 1)

['Bakersfield', 11]
['Beale', 11]
['Castle', 11]
['Fresno', 13]
['RedBluff', 13]
['Redding', 11]
['Sacramento', 13]
['Stockton', 11]
['TravisAFB', 11]


In [30]:
dataL = pd.DataFrame()
for fileName in fileList : 
    
    indexNum = 6
    if(not fileName.startswith(city)):
        varName = fileName[0:fileName.find('_')]
    else :
        varName = fileName[::-1][0:fileName[::-1].find('_')][::-1][:-4]
        if (varName == 'FogData') :
            indexNum = 0
            
    dataL = pd.concat([dataL, pd.read_csv(fileName, index_col=indexNum)], axis = 1)
    

In [31]:
dataL

,November,December,January,February,March,average,November,December,January,February,...,January,February,March,average,November,December,January,February,March,average
1940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1943,4.274,2.274,3.609,4.163,3.262,3.516,7.829,6.793,4.357,5.253,...,12.241,13.612,15.229,13.186,3.588,4.744,4.499,5.402,6.492,4.945
1944,6.536,5.648,3.648,4.500,5.318,5.130,4.200,2.288,3.210,6.192,...,9.505,14.326,NaN,NaN,4.033,3.229,3.570,5.598,5.418,4.370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,5.767,0.579,3.330,4.664,6.047,4.077,4.235,6.344,4.681,5.426,...,14.436,15.582,15.296,14.399,3.281,3.338,2.991,3.845,4.001,3.491
2012,8.861,5.724,2.521,3.937,8.190,5.847,2.858,2.353,3.322,5.184,...,14.634,15.589,15.739,14.794,2.761,3.062,2.636,3.618,4.158,3.247
2013,4.767,-0.747,1.282,8.740,9.039,4.616,7.905,6.989,8.198,2.610,...,15.887,15.012,15.817,15.509,3.463,2.766,2.934,3.783,4.438,3.477
2014,8.699,8.080,5.503,8.045,8.752,7.816,3.933,3.190,3.032,4.293,...,10.922,14.186,15.803,13.508,2.647,3.081,2.044,3.392,3.728,2.978
